In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"./Aleppo2017_processed.csv",encoding='utf-8')
df.head()

,id,time,gl
0,183,2015-05-16 05:35:41,162.0
1,183,2015-05-16 05:30:41,164.0
2,183,2015-05-16 05:25:41,168.0
3,183,2015-05-16 05:20:41,169.0
4,183,2015-05-16 05:15:41,170.0


In [2]:
ID = df['id'].value_counts().index
print(ID)

Index([263,  77, 193, 277, 229, 245, 155, 251, 111, 164,
       ...
       128, 264, 186, 162, 249,  39,  52, 266, 223, 289],
      dtype='int64', name='id', length=226)


In [3]:
import datetime
for i in ID[:100]:
    print("开始筛选ID为{}患者的数据-------------".format(i))
    
    #获取每个被试的数据
    data = df[df['id'] == i].copy()
    
    #对每个被试的数据根据时间进行排序
    #1、先将时间列设成datetime类型
    data['time'] = pd.to_datetime(data['time']) 
    
    #2、根据时间排序
    data = data.sort_values(by='time')
    
    #3、统计每个被试每天中有多少血糖数据
        #3.1 提取时间的年月日信息, 一天中的血糖数据的年月日信息相同
    data['date'] = data['time'].dt.date
        #3.2 统计相同的年月日中有多少血糖数据
    daily_data_count = data.groupby('date')['gl'].count()
    
    #4、筛选数据
        #4.1 连续7天
        #4.2 每天的数据量大于260
    continuous_days = 0    #记录连续的天数
    target_dates = []    #存放筛选出的数据(time)
    
    #此段for循环找出参与者第一个连续7天，且每天血糖数据大于260的日期

    for date,count in daily_data_count.items():
  
        if continuous_days >= 7:
            break
            
        else: 
            consecutive_count = 0
            label = 1
            data_of_day = data[data['date'] == date]
            for k in range(24):
                hour_data = data_of_day[data_of_day['time'].dt.hour == k].copy()    #某些小时可能没有数据
                
                if hour_data.empty:
                    consecutive_count += 1
                    if consecutive_count >= 2:
                        label = 0
                else:
                    consecutive_count = 0
                        
            if count > 240 and label:
                target_dates.append(date)
                continuous_days += 1

            else:
                continuous_days = 0
                target_dates = []
                
    print(target_dates)
    #5、处理target_dates中记录的日期对应的数据
    #5.1 获取处理target_dates中每天的数据
    total_data = []
    if target_dates:
        for j in target_dates:
            data_of_day = data[data['date'] == j]

            #5.2 如果这天中的数据满足288条则直接写入csv,如果不满足288则需要进行插值处理
            if data_of_day.shape[0] == 288:

                print("{}这天测够了288条数据".format(j))

                data_of_day = data_of_day.set_index('time')
                
                total_data.append(data_of_day)

            else:
                #先创建一个空的dataframe，将每小时的数据拼接到空的dataframe
                columns_name = ['id','gl','date']
                day_of_data = pd.DataFrame(columns=columns_name)
                day_of_data.index.name = 'time'

                print("{} 这天的血糖记录数不满足288".format(j))

                print("day_data.shape",data_of_day.shape)

                #设置秒为0
                data_of_day.loc[:,'time'] = data_of_day['time'].apply(lambda dt: dt.replace(second=0))

                for k in range(24):
                    hour_data = data_of_day[data_of_day['time'].dt.hour == k].copy()    #某些小时可能没有数据

                    if hour_data.empty:
                        print("数据在 {}-{}  小时时间范围内为空".format(k,k+1))

                        start_date = datetime.datetime.combine(j, datetime.time(k, 0))

                        full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                        data_empty = {'id':[i] * len(full_time_index),'gl':np.nan * len(full_time_index),'date':np.nan * len(full_time_index)}

                        data_day_padding = pd.DataFrame(data_empty, index=full_time_index)

                        day_of_data = pd.concat((day_of_data,data_day_padding),axis=0)

                    else:
                        minute = hour_data.iloc[0]['time'].minute % 5

                        # 设置一小时内的起始时间
                        start_date = hour_data.iloc[0]['time'].replace(hour=k, minute=minute)
        #                 print(start_date)

                        # 使用完整的时间序列重新索引
                        full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                        #删除重复时间的记录
                        data_unique_times = hour_data[~hour_data['time'].duplicated(keep='first')]

                        # 将时间戳设置为索引
                        data_unique_times = data_unique_times.set_index('time')

                        # 使用完整的时间序列重新索引
                        hour_data_interpolated = data_unique_times.reindex(index=full_time_index)

                        hour_data_interpolated['id'] = i
                        
                        day_of_data = pd.concat((day_of_data, hour_data_interpolated),axis=0)

                total_data.append(day_of_data)
        total_data = pd.concat(total_data)
        total_data['gl'] = total_data['gl'].interpolate().round()
        total_data.to_csv(r"C:\Users\admin\Desktop\week_of_data4.csv", mode='a', header=False)   

开始筛选ID为263患者的数据-------------
[datetime.date(2015, 5, 20), datetime.date(2015, 5, 21), datetime.date(2015, 5, 22), datetime.date(2015, 5, 23), datetime.date(2015, 5, 24), datetime.date(2015, 5, 25), datetime.date(2015, 5, 26)]
2015-05-20 这天的血糖记录数不满足288
day_data.shape (270, 4)
2015-05-21这天测够了288条数据
2015-05-22 这天的血糖记录数不满足288
day_data.shape (417, 4)
2015-05-23 这天的血糖记录数不满足288
day_data.shape (367, 4)
2015-05-24 这天的血糖记录数不满足288
day_data.shape (279, 4)
2015-05-25 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-05-26 这天的血糖记录数不满足288
day_data.shape (289, 4)
开始筛选ID为77患者的数据-------------
[datetime.date(2015, 5, 16), datetime.date(2015, 5, 17), datetime.date(2015, 5, 18), datetime.date(2015, 5, 19), datetime.date(2015, 5, 20), datetime.date(2015, 5, 21), datetime.date(2015, 5, 22)]
2015-05-16这天测够了288条数据
2015-05-17 这天的血糖记录数不满足288
day_data.shape (282, 4)
2015-05-18这天测够了288条数据
2015-05-19 这天的血糖记录数不满足288
day_data.shape (284, 4)
2015-05-20这天测够了288条数据
2015-05-21 这天的血糖记录数不满足288
day_data.shape (284, 4)
2015-05-22 

数据在 12-13  小时时间范围内为空
开始筛选ID为23患者的数据-------------
[datetime.date(2015, 5, 18), datetime.date(2015, 5, 19), datetime.date(2015, 5, 20), datetime.date(2015, 5, 21), datetime.date(2015, 5, 22), datetime.date(2015, 5, 23), datetime.date(2015, 5, 24)]
2015-05-18 这天的血糖记录数不满足288
day_data.shape (284, 4)
2015-05-19 这天的血糖记录数不满足288
day_data.shape (285, 4)
2015-05-20这天测够了288条数据
2015-05-21这天测够了288条数据
2015-05-22 这天的血糖记录数不满足288
day_data.shape (266, 4)
数据在 13-14  小时时间范围内为空
2015-05-23 这天的血糖记录数不满足288
day_data.shape (293, 4)
2015-05-24 这天的血糖记录数不满足288
day_data.shape (293, 4)
开始筛选ID为232患者的数据-------------
[datetime.date(2015, 5, 5), datetime.date(2015, 5, 6), datetime.date(2015, 5, 7), datetime.date(2015, 5, 8), datetime.date(2015, 5, 9), datetime.date(2015, 5, 10), datetime.date(2015, 5, 11)]
2015-05-05 这天的血糖记录数不满足288
day_data.shape (287, 4)
2015-05-06 这天的血糖记录数不满足288
day_data.shape (280, 4)
2015-05-07 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-05-08 这天的血糖记录数不满足288
day_data.shape (285, 4)
2015-05-09 这天的血糖记录

[datetime.date(2015, 6, 8), datetime.date(2015, 6, 9), datetime.date(2015, 6, 10), datetime.date(2015, 6, 11), datetime.date(2015, 6, 12), datetime.date(2015, 6, 13), datetime.date(2015, 6, 14)]
2015-06-08 这天的血糖记录数不满足288
day_data.shape (282, 4)
2015-06-09 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-06-10 这天的血糖记录数不满足288
day_data.shape (293, 4)
2015-06-11 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-06-12 这天的血糖记录数不满足288
day_data.shape (292, 4)
2015-06-13 这天的血糖记录数不满足288
day_data.shape (293, 4)
2015-06-14 这天的血糖记录数不满足288
day_data.shape (263, 4)
开始筛选ID为187患者的数据-------------
[datetime.date(2015, 5, 11), datetime.date(2015, 5, 12), datetime.date(2015, 5, 13), datetime.date(2015, 5, 14), datetime.date(2015, 5, 15), datetime.date(2015, 5, 16), datetime.date(2015, 5, 17)]
2015-05-11 这天的血糖记录数不满足288
day_data.shape (267, 4)
数据在 0-1  小时时间范围内为空
2015-05-12 这天的血糖记录数不满足288
day_data.shape (270, 4)
2015-05-13 这天的血糖记录数不满足288
day_data.shape (279, 4)
2015-05-14 这天的血糖记录数不满足288
day_data.shape (283, 4)
2015-05-15

2015-05-29 这天的血糖记录数不满足288
day_data.shape (252, 4)
开始筛选ID为81患者的数据-------------
[datetime.date(2015, 5, 23), datetime.date(2015, 5, 24), datetime.date(2015, 5, 25), datetime.date(2015, 5, 26), datetime.date(2015, 5, 27), datetime.date(2015, 5, 28), datetime.date(2015, 5, 29)]
2015-05-23 这天的血糖记录数不满足288
day_data.shape (292, 4)
2015-05-24 这天的血糖记录数不满足288
day_data.shape (292, 4)
2015-05-25 这天的血糖记录数不满足288
day_data.shape (292, 4)
2015-05-26 这天的血糖记录数不满足288
day_data.shape (293, 4)
2015-05-27这天测够了288条数据
2015-05-28 这天的血糖记录数不满足288
day_data.shape (293, 4)
2015-05-29 这天的血糖记录数不满足288
day_data.shape (269, 4)
数据在 9-10  小时时间范围内为空
开始筛选ID为109患者的数据-------------
[datetime.date(2015, 5, 23), datetime.date(2015, 5, 24), datetime.date(2015, 5, 25), datetime.date(2015, 5, 26), datetime.date(2015, 5, 27), datetime.date(2015, 5, 28), datetime.date(2015, 5, 29)]
2015-05-23 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-05-24 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-05-25 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-05

数据在 8-9  小时时间范围内为空
开始筛选ID为46患者的数据-------------
[datetime.date(2015, 6, 26), datetime.date(2015, 6, 27), datetime.date(2015, 6, 28), datetime.date(2015, 6, 29), datetime.date(2015, 6, 30), datetime.date(2015, 7, 1), datetime.date(2015, 7, 2)]
2015-06-26 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-06-27 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-06-28 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-06-29 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-30 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-07-01 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-07-02 这天的血糖记录数不满足288
day_data.shape (264, 4)
数据在 11-12  小时时间范围内为空
开始筛选ID为61患者的数据-------------
[datetime.date(2015, 5, 30), datetime.date(2015, 5, 31), datetime.date(2015, 6, 1), datetime.date(2015, 6, 2), datetime.date(2015, 6, 3), datetime.date(2015, 6, 4), datetime.date(2015, 6, 5)]
2015-05-30 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-05-31 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-01 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-02 这天的血糖记

2015-05-25 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-05-26 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-05-27 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-05-28 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-05-29 这天的血糖记录数不满足288
day_data.shape (266, 4)
数据在 11-12  小时时间范围内为空
开始筛选ID为119患者的数据-------------
[datetime.date(2015, 6, 6), datetime.date(2015, 6, 7), datetime.date(2015, 6, 8), datetime.date(2015, 6, 9), datetime.date(2015, 6, 10), datetime.date(2015, 6, 11), datetime.date(2015, 6, 12)]
2015-06-06 这天的血糖记录数不满足288
day_data.shape (279, 4)
2015-06-07 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-08 这天的血糖记录数不满足288
day_data.shape (294, 4)
2015-06-09 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-10 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-11 这天的血糖记录数不满足288
day_data.shape (286, 4)
2015-06-12 这天的血糖记录数不满足288
day_data.shape (259, 4)
数据在 8-9  小时时间范围内为空
开始筛选ID为32患者的数据-------------
[datetime.date(2015, 5, 31), datetime.date(2015, 6, 1), datetime.date(2015, 6, 2), datetime.date(2015, 6, 3), 

[datetime.date(2015, 6, 3), datetime.date(2015, 6, 4), datetime.date(2015, 6, 5), datetime.date(2015, 6, 6), datetime.date(2015, 6, 7), datetime.date(2015, 6, 8), datetime.date(2015, 6, 9)]
2015-06-03 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-06-04 这天的血糖记录数不满足288
day_data.shape (291, 4)
2015-06-05 这天的血糖记录数不满足288
day_data.shape (289, 4)
2015-06-06 这天的血糖记录数不满足288
day_data.shape (293, 4)
2015-06-07 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-06-08 这天的血糖记录数不满足288
day_data.shape (290, 4)
2015-06-09 这天的血糖记录数不满足288
day_data.shape (249, 4)
数据在 16-17  小时时间范围内为空
开始筛选ID为24患者的数据-------------
[datetime.date(2015, 6, 10), datetime.date(2015, 6, 11), datetime.date(2015, 6, 12), datetime.date(2015, 6, 13), datetime.date(2015, 6, 14), datetime.date(2015, 6, 15), datetime.date(2015, 6, 16)]
2015-06-10 这天的血糖记录数不满足288
day_data.shape (284, 4)
2015-06-11 这天的血糖记录数不满足288
day_data.shape (284, 4)
2015-06-12 这天的血糖记录数不满足288
day_data.shape (422, 4)
2015-06-13 这天的血糖记录数不满足288
day_data.shape (542, 4)
2015-06-14 这天的